<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Tracing and Evaluating a Structured Data Extraction Service</h1>

In this tutorial, you will:

- Use Anthropic's [tool calling](https://docs.anthropic.com/en/docs/build-with-claude/tool-use) to perform structured data extraction: the task of transforming unstructured input (e.g., user requests in natural language) into structured format (e.g., tabular format),
- Instrument your Anthropic client to record trace data in [OpenInference tracing](https://github.com/Arize-ai/open-inference-spec/blob/main/trace/spec/traces.md) format,
- Inspect the traces and spans of your application to visualize your trace data,
- Export your trace data to run an evaluation on the quality of your structured extractions.

## Background

One powerful feature of the Anthropic API is tool use (function calling), wherein a user describes the signature and arguments of one or more functions to the Anthropic API via a JSON Schema and natural language descriptions, and the LLM decides when to call each function and provides argument values depending on the context of the conversation. In addition to its primary purpose of integrating function inputs and outputs into a sequence of chat messages, function calling is also useful for structured data extraction, since you can specify a "function" that describes the desired format of your structured output. Structured data extraction is useful for a variety of purposes, including ETL or as input to another machine learning model such as a recommender system.

While it's possible to produce structured output without using function calling via careful prompting, function calling is more reliable at producing output that conforms to a particular format. For more details on Anthropic's function calling API, see the [Anthropic documentation](https://docs.anthropic.com/en/home).

Let's get started!

ℹ️ This notebook requires an Anthropic API key.

## 1. Install Dependencies and Import Libraries

Install dependencies.

In [ ]:
!pip install anthropic arize-phoenix jsonschema openinference-instrumentation-anthropic

Import libraries.

In [ ]:
import os
from getpass import getpass
from typing import Any, Dict

import pandas as pd

import phoenix as px

pd.set_option("display.max_colwidth", None)

## 2. Configure Your Anthropic API Key and Instantiate Your Anthropic Client

Set your Anthropic API key if it is not already set as an environment variable.

In [ ]:
import anthropic

if not (anthropic_api_key := os.getenv("ANTHROPIC_API_KEY")):
    anthropic_api_key = getpass("🔑 Enter your Anthropic API key: ")
client = anthropic.Anthropic(api_key=anthropic_api_key)

## 3. Instrument Your Anthropic Client

Instrument your Anthropic client with a tracer that emits telemetry data in OpenInference format. [OpenInference](https://arize-ai.github.io/open-inference-spec/trace/) is an open standard for capturing and storing LLM application traces that enables LLM applications to seamlessly integrate with LLM observability solutions such as Phoenix.

In [ ]:
from openinference.instrumentation.anthropic import AnthropicInstrumentor

from phoenix.otel import register

tracer_provider = register(project_name="anthropic-tools")
AnthropicInstrumentor().instrument(tracer_provider=tracer_provider)

## 4. Run Phoenix in the Background

Launch Phoenix as a background session to collect the trace data emitted by your instrumented OpenAI client.

In [ ]:
(session := px.launch_app()).view()

## 5. Extract Your Structured Data

We'll extract structured data from the following list of ten travel requests.

In [ ]:
travel_requests = [
    "Can you recommend a luxury hotel in Tokyo with a view of Mount Fuji for a romantic honeymoon?",
    "I'm looking for a mid-range hotel in London with easy access to public transportation for a solo backpacking trip. Any suggestions?",
    "I need a budget-friendly hotel in San Francisco close to the Golden Gate Bridge for a family vacation. What do you recommend?",
    "Can you help me find a boutique hotel in New York City with a rooftop bar for a cultural exploration trip?",
    "I'm planning a business trip to Tokyo and I need a hotel near the financial district. What options are available?",
    "I'm traveling to London for a solo vacation and I want to stay in a trendy neighborhood with great shopping and dining options. Any recommendations for hotels?",
    "I'm searching for a luxury beachfront resort in San Francisco for a relaxing family vacation. Can you suggest any options?",
    "I need a mid-range hotel in New York City with a fitness center and conference facilities for a business trip. Any suggestions?",
    "I'm looking for a budget-friendly hotel in Tokyo with easy access to public transportation for a backpacking trip. What do you recommend?",
    "I'm planning a honeymoon in London and I want a luxurious hotel with a spa and romantic atmosphere. Can you suggest some options?",
]

The Anthropic API uses [JSON Schema](https://json-schema.org/) and natural language descriptions to specify the signature of a function to call. In this case, we'll describe a function to record the following attributes of the unstructured text input:

- **location:** The desired destination,
- **budget_level:** A categorical budget preference,
- **purpose:** The purpose of the trip.

The use of JSON Schema enables us to define the type of each field in the output and even enumerate valid values in the case of categorical outputs. Anthropic function calling can thus be used for tasks that might previously have been performed by named-entity recognition (NER) and/ or classification models.

In [ ]:
tool_schema = {
    "name": "record_travel_request_attributes",
    "description": "Records the attributes of a travel request",
    "input_schema": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": 'The desired destination location. Use city, state, and country format when possible. If no destination is provided, return "not_stated".',
            },
            "budget_level": {
                "type": "string",
                "enum": ["low", "medium", "high", "not_stated"],
                "description": 'The desired budget level. If no budget level is provided, return "not_stated".',
            },
            "purpose": {
                "type": "string",
                "enum": ["business", "pleasure", "other", "not_stated"],
                "description": 'The purpose of the trip. If no purpose is provided, return "not_stated".',
            },
        },
        "required": ["location", "budget_level", "purpose"],
    },
}

system_message = (
    "You are an assistant that parses and records the attributes of a user's travel request."
)


def extract_raw_travel_request_attributes_string(
    travel_request: str,
    tool_schema: Dict[str, Any],
    system_message: str,
    client: client,
    model: str = "claude-3-5-sonnet-20240620",
) -> str:
    response = client.messages.create(
        model=model,
        max_tokens=1024,
        messages=[
            {"role": "user", "content": travel_request},
        ],
        system=system_message,
        tools=[tool_schema],
        # By default, the LLM will choose whether or not to call a function given the conversation context.
        # The line below forces the LLM to call the function so that the output conforms to the schema.
        tool_choice={"type": "tool", "name": "record_travel_request_attributes"},
    )
    return response.content[0].input

Run the extractions.

In [ ]:
raw_travel_attributes_column = []
for travel_request in travel_requests:
    print("Travel request:")
    print("==============")
    print(travel_request)
    print()
    raw_travel_attributes = extract_raw_travel_request_attributes_string(
        travel_request, tool_schema, system_message, client
    )
    raw_travel_attributes_column.append(raw_travel_attributes)
    print("Raw Travel Attributes:")
    print("=====================")
    print(raw_travel_attributes)
    print()
    print()

Your trace data should appear in real-time in the Phoenix UI.

In [ ]:
print(f"🔥🐦 Open the Phoenix UI if you haven't already: {session.url}")

# 6. Export and Evaluate Your Trace Data

Your OpenInference trace data is collected by Phoenix and can be exported to a pandas dataframe for further analysis and evaluation.

In [ ]:
trace_ds = px.Client().get_trace_dataset(project_name="anthropic-tools")
trace_ds.get_spans_dataframe().head()

In [ ]:
trace_ds.save()

## 7. Recap

Congrats! In this tutorial, you:

- Built a service to perform structured data extraction on unstructured text using Anthropic function calling
- Instrumented your service with an OpenInference tracer
- Examined your telemetry data in Phoenix

Check back soon for tips on evaluating the performance of your service using LLM evals.